First, import needed modules

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
import os

Initalize variables

In [5]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [6]:
data = pd.read_csv(f'{path}/data/btw_treatment.csv', encoding = 'UTF-8-SIG', sep=',')
#data = data.set_index(['AGS', 'year'])
data.head(10)

,AGS,GEN,year,post_2005,post_2009,post_2013,post_2017,Land,Erststimme,Zweitstimme,...,treated_30*post_2013,treated_30*post_2017,treated_50*post_2005,treated_50*post_2009,treated_50*post_2013,treated_50*post_2017,treated_100*post_2005,treated_100*post_2009,treated_100*post_2013,treated_100*post_2017
0,1001000,Flensburg,2002.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001000,Flensburg,2002.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001000,Flensburg,2005.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1001000,Flensburg,2005.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1001000,Flensburg,2009.0,1.0,1.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5,1001000,Flensburg,2009.0,1.0,1.0,0.0,0.0,1.0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
6,1001000,Flensburg,2013.0,1.0,1.0,1.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
7,1001000,Flensburg,2013.0,1.0,1.0,1.0,0.0,1.0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
8,1001000,Flensburg,2017.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
9,1001000,Flensburg,2017.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0


Split data in Erst- and Zweitstimme

In [7]:
erst = data[data['Erststimme'] == 1]
zweit = data[data['Zweitstimme'] == 1]
erst.head(10)

,AGS,GEN,year,post_2005,post_2009,post_2013,post_2017,Land,Erststimme,Zweitstimme,...,treated_30*post_2013,treated_30*post_2017,treated_50*post_2005,treated_50*post_2009,treated_50*post_2013,treated_50*post_2017,treated_100*post_2005,treated_100*post_2009,treated_100*post_2013,treated_100*post_2017
0,1001000,Flensburg,2002.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001000,Flensburg,2005.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1001000,Flensburg,2009.0,1.0,1.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
6,1001000,Flensburg,2013.0,1.0,1.0,1.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
8,1001000,Flensburg,2017.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
10,1002000,Kiel,2002.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,1002000,Kiel,2005.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,1002000,Kiel,2009.0,1.0,1.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,1002000,Kiel,2013.0,1.0,1.0,1.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,1002000,Kiel,2017.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Statsmodels

DiD for primary vote, treatment 50 in 2013

In [11]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2013', data=erst).fit(cov_type='cluster', cov_kwds={'groups': np.array(erst[['AGS', 'year']])}) #locals converts string to variable name
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Primary Vote, Municipalities within 50 km of a powerline project, treatment year 2013')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/erst_2013.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

DiD for primary vote, treatment 50 in 2017

In [12]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2017', data=erst).fit(cov_type='cluster', cov_kwds={'groups': np.array(erst[['AGS', 'year']])})
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Primary Vote, Municipalities within 50 km of a powerline project, treatment year 2017')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/erst_2017.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

DiD for Secondary vote, treatment 50 in 2013

In [13]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2013', data=zweit).fit(cov_type='cluster', cov_kwds={'groups': np.array(zweit[['AGS', 'year']])})
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Secondary Vote, Municipalities within 50 km of a powerline project, treatment year 2013')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/zweit_2013.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

DiD for seconday vote, treatment 50 in 2017

In [34]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2017', data=zweit).fit(cov_type='cluster', cov_kwds={'groups': np.array(zweit[['AGS', 'year']])}) #np.array(zweit[['AGS', 'year']])
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Secondary Vote, Municipalities within 50 km of a powerline project, treatment year 2017')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/zweit_2017.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

Placebo DiD for treatments in 2005 and 2009

Erststimme post 2005

In [14]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2005', data=erst).fit(cov_type='cluster', cov_kwds={'groups': np.array(erst[['AGS', 'year']])})
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Secondary Vote, Municipalities within 50 km of a powerline project, treatment year 2005')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/erst_2005.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

Erststimme post 2009

In [15]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2009', data=erst).fit(cov_type='cluster', cov_kwds={'groups': np.array(erst[['AGS', 'year']])})
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Secondary Vote, Municipalities within 50 km of a powerline project, treatment year 2009')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/erst_2009.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

Zweitstimme post 2005

In [16]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2005', data=zweit).fit(cov_type='cluster', cov_kwds={'groups': np.array(zweit[['AGS', 'year']])}) # zweit['AGS']
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Secondary Vote, Municipalities within 50 km of a powerline project, treatment year 2005')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/zweit_2005.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

Zweitstimme post 2009

In [17]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2009', data=zweit).fit(cov_type='cluster', cov_kwds={'groups': np.array(zweit[['AGS', 'year']])}) # zweit['AGS']
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Secondary Vote, Municipalities within 50 km of a powerline project, treatment year 2009')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/zweit_2009.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())